In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

In [3]:
grd = pd.read_csv("../data/graphene_data_final.csv")

In [32]:
X, Y = grd[['Graphene_percentage', 'FEED', 'RPM', 'DOC']], grd['MRR_gm_per_sec']
Y2 = grd['Ra']
X2 = grd[['FEED','DOC']]

In [33]:
X_train2, X_test2, Y_train3, Y_test3 = train_test_split(X2, Y, test_size=0.3, random_state=33)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train, X_test, Y2_train, Y2_test = train_test_split(X, Y2, test_size=0.3, random_state=16)

In [6]:
rfr=RandomForestRegressor(random_state=42)

In [7]:
param_grid = { 
    'n_estimators': [30,40,50,70,100,150,200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [1,2,3,4,5,6,7,8,9],
}

In [8]:
CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
CV_rfr.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [30, 40, 50, 70, 100, 150, 200]})

In [9]:
CV_rfr.score(X_train, Y_train)

0.4831339819133369

In [10]:
CV_rfr.score(X_test, Y_test)

-0.003600504622853551

In [26]:
rfr2=RandomForestRegressor(n_estimators=100,max_depth=11,min_samples_split=8).fit(X_train, Y_train)

In [36]:
rfr4.score(X_test2, Y_test3)

0.9419066691122864

In [27]:
rfr2.score(X_test, Y_test)

-0.14445240523684588

In [28]:
rfr2.score(X_train, Y_train)

0.472516874418621

In [29]:
rfr3=RandomForestRegressor()

In [30]:
cv_results = cross_validate(rfr3, X, Y, cv=4)
sorted(cv_results.keys())

['fit_time', 'score_time', 'test_score']

In [31]:
cv_results['test_score']

array([0.85208953, 0.79366659, 0.88600638, 0.883734  ])

In [41]:
rfr4=RandomForestRegressor(n_estimators=1200,
                           max_depth=11,
                           min_samples_leaf=2,
                           min_samples_split=8).fit(X_train2, Y_train3)

In [42]:
rfr4.score(X_test2, Y_test3)

0.9420591137605021